In [1]:
import sqlite3
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np



In [3]:
DB_PATH = "../anaslo_02/anaslo_02.db"
SPREADSHEET_ID = "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8"

In [4]:
# スプレッドシート認証設定
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
jsonf = r"C:\python\dataOnline\anaslo_02\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)

In [5]:
# Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
conn.close()

print(tables)

[('halls',), ('models',), ('prefectures',), ('results',), ('sqlite_sequence',)]


In [6]:
# 検索キーワード
HALL_WORD = "EXA_FIRST"

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT hall_id, name FROM halls WHERE name LIKE ?", ('%' + HALL_WORD + '%',))
results = cursor.fetchall()

# 結果表示
if results:
    print(f"🔍 '{HALL_WORD}' を含むホール名の検索結果:")
    for hall_id, hall_name in results:
        print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
else:
    print(f"❌ '{HALL_WORD}' を含むホール名は見つかりませんでした。")
    
conn.close()

🔍 'EXA_FIRST' を含むホール名の検索結果:
 - hall_name: EXA FIRST, hall_id: 108


In [7]:
query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
  AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

conn = sqlite3.connect(DB_PATH)
df = pd.read_sql_query(query, conn, params=(hall_name,))

df['date'] = pd.to_datetime(df['date'])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["day"] = df["date"].dt.day
df["month"] = df["date"].dt.month
df["weekday"] = df["date"].dt.weekday

conn.close()

In [8]:
df.shape

(51376, 14)

In [10]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)
df.shape
# DataFrame をシートに書き込む
sheet_name = "DATA"
spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(sheet_name)
sheet.clear()
set_with_dataframe(sheet, df)
print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

✅ シート 'DATA' に DataFrame を書き込みました！


In [11]:
csv_file = f"../anaslo_02/out/{hall_name}_analysis_data.csv"
df.to_csv(csv_file, index=False, encoding="utf-8-sig")

In [13]:
print(df.shape)
df.head()

(51376, 14)


,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Total_rate,day,month,weekday
0,EXA FIRST,2025-04-06,マイジャグラーV,1001,6813,29,21,879,234.9,324.4,136.3,6,4,6
1,EXA FIRST,2025-04-06,マイジャグラーV,1002,5540,26,20,1759,213.1,277.0,120.4,6,4,6
2,EXA FIRST,2025-04-06,マイジャグラーV,1003,2892,13,6,415,222.5,482.0,152.2,6,4,6
3,EXA FIRST,2025-04-06,マイジャグラーV,1004,2562,6,6,-700,427.0,427.0,213.5,6,4,6
4,EXA FIRST,2025-04-06,マイジャグラーV,1005,4940,19,11,-562,260.0,449.1,164.7,6,4,6


In [14]:
for model in df["model_name"].unique():
    print(f"機種名: {model}")
    df_tmp = df[df["model_name"] == model]
    medals = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="day", values="medals", aggfunc="sum", margins=True, margins_name="Total")
    csv_file = f"../anaslo_02/out/{hall_name}_{model}_medals_by_day.csv"
    medals.to_csv(csv_file, encoding="utf-8-sig")
    
    game = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="day", values="game", aggfunc="sum", margins=True, margins_name="Total")
    rb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="day", values="RB", aggfunc="sum", margins=True, margins_name="Total")
    rb_rate = (game / rb).round(1)
    csv_file = f"../anaslo_02/out/{hall_name}_{model}_rb_rate_by_day.csv"
    rb_rate.to_csv(csv_file, encoding="utf-8-sig")
    
    display(medals.head())

機種名: マイジャグラーV


day                    1     2     3     4      5     6     7     8     9  \
model_name unit_no                                                          
マイジャグラーV   1001     9917 -3144   916  1317  -1476 -5136 -1737 -7764  -970   
           1002     7762  5508  2598  1647   3488  4011  1606  2891 -2044   
           1003     9466  6246  6696 -2319  12364 -5080  1998  -712 -1454   
           1004      223  2998  6149 -1646   -722 -2959  3547  4362  -942   
           1005     9875  3408 -1364  8560   2220  -762  2063 -2678 -1377   

day                   10  ...    23    24    25     26     27    28    29  \
model_name unit_no        ...                                               
マイジャグラーV   1001    -6080  ...  2653 -3110 -3735  -3835   1699 -3036  -682   
           1002     4820  ... -4778  3438  3752    633   1333  2654 -1189   
           1003    -1480  ...   535  3094 -4259  11006    342 -4250 -1667   
           1004    -2108  ...  8654 -3543  -594   3743  10908 -3272  4132   
           1005     1525  ... -1182  3200 -3588   4024    628  2823 -3939   

day                   30    31  Total  
model_name unit_no                     
マイジャグラーV   1001     4838  -453 -31132  
           1002     1741  1395  50951  
           1003     1327 -4094  58216  
           1004    -1990  8418  44403  
           1005     1127  4205  45655  

[5 rows x 32 columns]

機種名: ファンキージャグラー2


day                     1     2     3     4     5     6     7     8     9  \
model_name  unit_no                                                         
ファンキージャグラー2 1051    -4152  2794 -5407 -1518 -4534 -6127  1645  -518 -5539   
            1052      388  1035   899 -2971  4096 -3932 -4627  2604 -5048   
            1053    -4557 -2276  2019  -999 -6069  7679  -207  4323 -2438   
            1054     1889 -4169  1041  1581   244    85 -4626  3824 -1057   
            1055    -5953 -4264  4538 -1994 -2187  -196 -3131  1758 -2266   

day                    10  ...    23    24    25    26    27    28    29  \
model_name  unit_no        ...                                             
ファンキージャグラー2 1051      593  ...   517 -6934  4203  3828 -8502 -4280 -1887   
            1052    -4548  ... -1178 -2323   383  -560  -657  1328 -1820   
            1053    -3927  ... -3743   394  -948  4945   416   889 -2223   
            1054    -3464  ...  3062  -218 -4450  1673  3355   981  -603   
            1055     2449  ...  -129  2201  1868 -2703 -3140 -3228 -1706   

day                    30    31  Total  
model_name  unit_no                     
ファンキージャグラー2 1051    -1781  1472 -73886  
            1052      844 -3199   8366  
            1053    -1556  4829  23100  
            1054     -824  1668   3439  
            1055     1859  1683 -21507  

[5 rows x 32 columns]

機種名: ミスタージャグラー


...

day                    1     2     3     4     5     6     7     8     9  \
model_name unit_no                                                         
ジャグラーガールズ  1100     -435  -905   665 -2335   585   938  -902   262   256   
           1101     1747  1685  1039   221   829 -2176  -127  1014   267   
           1102    -3164  -855  4368  -312 -2183 -2107  1462 -1432 -2208   
           1103    -1817  2112 -2201 -3152  -727   976   359   856  -807   
           1104     2617  -865 -2443  -414 -3118  4752    59 -2600 -2774   

day                   10  ...    23    24    25    26    27    28    29    30  \
model_name unit_no        ...                                                   
ジャグラーガールズ  1100       88  ...  1380 -2374  1305  5908   145 -1615  1359    92   
           1101    -2508  ...   720   338   333   998  6171  3462 -1579 -1497   
           1102     -250  ... -2920 -1626  3070 -1847  4609  2157  -300   738   
           1103     -972  ... -1648 -1395  9618   862  2793 -3089   995 -1256   
           1104    -2645  ...   939  -765 -1459  1561  2053 -1171 -1099  1300   

day                   31  Total  
model_name unit_no               
ジャグラーガールズ  1100    -1756   -609  
           1101     2036  15855  
           1102    -1897   1066  
           1103       56  -4233  
           1104    -2053  -7711  

[5 rows x 32 columns]

機種名: ハッピージャグラーVIII


day                       1     2     3     4     7     8     9    10    11  \
model_name    unit_no                                                         
ハッピージャグラーVIII 1145     -697  -856 -1359 -1535  2479   676  1124 -1171  1700   
              1146      571   -18 -1300  -332  2921     9  -444  1076  -691   
              1147      238  -509 -1535  -150  2091   -21  -615   838  2268   
              1148    -1218  -591  -615  1771   912  1297   874  -715  1547   
              1149    -1112 -1065 -1265  -762  2718  -626  -179  -903  -809   

day                      12  ...    23    24    25    26   27    28    29  \
model_name    unit_no        ...                                            
ハッピージャグラーVIII 1145    -1688  ...    21 -1006   418   350  338  3259    56   
              1146      394  ...   665  -638  -591  1418 -374  2126   806   
              1147     -950  ... -1300  -226  3926   982  -44  -362  1771   
              1148     -538  ...   138  -432   544   -29  -76   350   979   
              1149      215  ...    41     3    21 -1135  -53  -559  -244   

day                      30    31  Total  
model_name    unit_no                     
ハッピージャグラーVIII 1145       15  1371   -446  
              1146     -479  -326   2678  
              1147     1053  -268   2230  
              1148     -715   -88   2022  
              1149     -526   650  -7506  

[5 rows x 30 columns]

機種名: SミスタージャグラーKK


day                      1    2     3     4     5     6    22    23    24  \
model_name   unit_no                                                        
SミスタージャグラーKK 1061      603 -959 -1206   500  2268  1241   362  1171  -615   
             1062     2497   -9  3259 -1441 -1041  -356   688 -1159 -1135   
             1063     2497  338  1665 -1159 -1606  -479  1229  1597   826   
             1064     -500 -735  -218   291  3753 -1747  2215  -853   347   
             1065      538 -774  2388   641  -735  1826 -2515  1685  1971   

day                     25    26    27    28    29    30    31  Total  
model_name   unit_no                                                   
SミスタージャグラーKK 1061     -653 -1429  3494 -1900  -947   641  1924   4495  
             1062     2232  -747   524  -650  -891  1241  -650   2362  
             1063    -2691  -503  2250 -1676   197   450    -3   2932  
             1064     1844  -488  -829  1241  -253 -1159  -606   2303  
             1065    -1606 -1359  1488 -1088 -1171    50  1029   2368